In [1]:
import pandas as pd

In [36]:
ev_df = pd.DataFrame({
    'year': [2022, 2022, 2023, 2023],
    'county': ['King', 'Spokane', 'King', 'Spokane'],
    'nEV': [5, 2, 6, 3],
    'efficiency_kwh_mi': [0.3, 0.3, 0.4, 0.4],
})
ev_df

,year,county,nEV,efficiency_kwh_mi
0,2022,King,5,0.3
1,2022,Spokane,2,0.3
2,2023,King,6,0.4
3,2023,Spokane,3,0.4


In [45]:
import numpy as np


In [46]:
people_df

,person_id,county
0,0,King
1,1,King
2,2,King
3,3,King
4,4,King
5,5,King
6,6,King
7,7,King
8,8,King
9,9,King


In [47]:
import pandas as pd
import numpy as np

# Sample dataframe with people and their home county
people_df = pd.DataFrame({
    'person_id': np.arange(20),
    'County': ['King']*10 + ['Spokane']*10,
})
# Population dataframe with county, year, and number of people to select
ev_df = pd.DataFrame({
    'Year': [2022, 2022, 2023, 2023],
    'County': ['King', 'Spokane', 'King', 'Spokane'],
    'nEV': [5, 2, 6, 3],
    'efficiency_kwh_mi': [0.3, 0.3, 0.4, 0.4],
})

selected_people = []
for _, row in ev_df.iterrows():
    county = row['County']
    year = row['Year']
    efficiency_kwh_mi = row['efficiency_kwh_mi']
    num_to_select = row['nEV']
    county_df = people_df[(people_df['County'] == county)]
    selected = county_df.sample(n=num_to_select, replace=False, random_state=42)
    # Add the year column to the selected dataframe
    selected['Year'] = year  
    # Add the efficiency column to the selected dataframe
    selected['efficiency_kwh_mi'] = efficiency_kwh_mi  
    selected_people.append(selected)

# Concatenate the selected people into a final dataframe
final_df = pd.concat(selected_people)


In [48]:
final_df   

,person_id,County,Year,efficiency_kwh_mi
8,8,King,2022,0.3
1,1,King,2022,0.3
5,5,King,2022,0.3
0,0,King,2022,0.3
7,7,King,2022,0.3
18,18,Spokane,2022,0.3
11,11,Spokane,2022,0.3
8,8,King,2023,0.4
1,1,King,2023,0.4
5,5,King,2023,0.4
